#credit card Fraud Detection
Business objective:
-Extremely imbalanced data
-accuracy is misleading
Solution Strategies:
-compare muktiple models
-use ROU-AUC for model selection
-track everything using mlflow

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import joblib

In [3]:
import mlflow
import mlflow.sklearn
import mlflow.xgboost

In [4]:
from sklearn.model_selection import train_test_split,RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

In [5]:
from sklearn.metrics import roc_auc_score,precision_recall_curve,classification_report,average_precision_score

In [6]:
from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier

<h1 style="color:red;font-size:40px">LOAD DATASET </h1>

In [7]:
df=pd.read_csv(r"C:\Users\badin\Downloads\archive (8)\creditcard.csv")
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [8]:
assert "Class" in df.columns, "Target column missing"
assert df.isnull().sum().sum()==0,"Missing values detected"

In [9]:
df.shape

(284807, 31)

In [10]:
df["Class"].value_counts(normalize=True)

Class
0    0.998273
1    0.001727
Name: proportion, dtype: float64

In [11]:
scaler=StandardScaler()


In [12]:
X=df.drop("Class",axis=1)
y=df["Class"]
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,stratify=y,random_state=42)

In [13]:
X_train["Amount"]=scaler.fit_transform(X_train[["Amount"]])
joblib.dump(scaler,"amount_scaler.pkl")

['amount_scaler.pkl']

In [ ]:
X_test["Amount"]=scaler.transform(X_test[["Amount"]])

In [21]:
smote=SMOTE(random_state=42)
X_train_res,y_train_res=smote.fit_resample(X_train,y_train)

pd.Series(y_train_res).value_counts()

Class
0    227451
1    227451
Name: count, dtype: int64

In [22]:
mlflow.set_experiment("Fraud_Detection")

<Experiment: artifact_location=('file:///c:/Users/badin/OneDrive/Desktop/Fraud '
 'Detection/Fraud-Detection/mlruns/1'), creation_time=1768663506312, experiment_id='1', last_update_time=1768663506312, lifecycle_stage='active', name='Fraud_Detection', tags={'mlflow.experimentKind': 'custom_model_development'}>

In [23]:
with mlflow.start_run(run_name="Logistic_Regression"):
    lr=LogisticRegression(max_iter=2000,solver="lbfgs")
    lr.fit(X_train_res,y_train_res)
    y_prob_lr=lr.predict_proba(X_test)[:,1]
    auc_lr=roc_auc_score(y_test,y_prob_lr)
    mlflow.log_params({"model":"LogisticRegression","solver":"lbfgs","max_iter":2000,"pca_used":True,"smote_used":True})
    mlflow.log_metric("rou_auc",auc_lr)
    mlflow.sklearn.log_model(sk_model=lr,artifact_path="model")
    auc_lr


c:\Users\badin\anaconda3\envs\nenv3\Lib\site-packages\sklearn\linear_model\_logistic.py:406: ConvergenceWarning: lbfgs failed to converge after 2000 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=2000).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
2026/01/20 11:55:20 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🏃 View run Logistic_Regression at: http://127.0.0.1:5000/#/experiments/1/runs/f3cca1cecf2e442e89bd3e953b05d09b
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1


In [24]:
with mlflow.start_run(run_name="RandomForest"):
    rf=RandomForestClassifier(n_estimators=200,random_state=42,n_jobs=-1)
    rf.fit(X_train_res,y_train_res)
    y_prob_rf=rf.predict_proba(X_test)[:,1]
    auc_rf=roc_auc_score(y_test,y_prob_rf)
    mlflow.log_param("model","RandomForest")
    mlflow.log_metric("roc_auc",auc_rf)
    mlflow.sklearn.log_model(sk_model=rf,artifact_path="model")
    print(auc_rf)

2026/01/20 11:57:56 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


0.9832555190759478
🏃 View run RandomForest at: http://127.0.0.1:5000/#/experiments/1/runs/83c6a84319364810995a5993990a9c69
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1


In [25]:
with mlflow.start_run(run_name="XGBoost"):
    xgb=XGBClassifier(n_estimators=300,max_depth=5,learning_rate=0.05,eval_metric="logloss",random_state=42)
    xgb.fit(X_train_res,y_train_res)
    y_prob_xgb=xgb.predict_proba(X_test)[:,1]
    auc_xgb=roc_auc_score(y_test,y_prob_xgb)
    mlflow.log_param("model","XGBoost")
    mlflow.log_metric("auc_roc",auc_rf)
    mlflow.xgboost.log_model(xgb,artifact_path="model")
    print(auc_xgb)

2026/01/20 11:58:28 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


0.9804996597682405
🏃 View run XGBoost at: http://127.0.0.1:5000/#/experiments/1/runs/e3507de6cc054738a0f93eddf8453faa
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1


In [26]:
print("LogisticRegression:",auc_lr)
print("RandomForest:",auc_rf)
print("XGBoost:",auc_xgb)

LogisticRegression: 0.9760194032593341
RandomForest: 0.9832555190759478
XGBoost: 0.9804996597682405


In [27]:
param_grid={"n_estimators":[200,300,400],"max_depth":[3,5,7],"learning_rate":[0.03,0.05],"subsample":[0.8],"colsample_bytree":[0.8]}

In [28]:
xgb_base=XGBClassifier(eval_metric="logloss",random_state=42)
search=RandomizedSearchCV(estimator=xgb_base,param_distributions=param_grid,n_iter=10,scoring="average_precision",cv=3,n_jobs=-1,verbose=1)


In [29]:
search.fit(X_train_res,y_train_res)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


,"estimator estimator: estimator objectAn object of that type is instantiated for each grid point.This is assumed to implement the scikit-learn estimator interface.Either estimator needs to provide a ``score`` function,or ``scoring`` must be passed.","XGBClassifier...ree=None, ...)"
,"param_distributions param_distributions: dict or list of dictsDictionary with parameters names (`str`) as keys and distributionsor lists of parameters to try. Distributions must provide a ``rvs``method for sampling (such as those from scipy.stats.distributions).If a list is given, it is sampled uniformly.If a list of dicts is given, first a dict is sampled uniformly, andthen a parameter is sampled using that dict as above.","{'colsample_bytree': [0.8], 'learning_rate': [0.03, 0.05], 'max_depth': [3, 5, ...], 'n_estimators': [200, 300, ...], ...}"
,"n_iter n_iter: int, default=10Number of parameter settings that are sampled. n_iter tradesoff runtime vs quality of the solution.",10
,"scoring scoring: str, callable, list, tuple or dict, default=NoneStrategy to evaluate the performance of the cross-validated model onthe test set.If `scoring` represents a single score, one can use:- a single string (see :ref:`scoring_string_names`);- a callable (see :ref:`scoring_callable`) that returns a single value;- `None`, the `estimator`'s :ref:`default evaluation criterion ` is used.If `scoring` represents multiple scores, one can use:- a list or tuple of unique strings;- a callable returning a dictionary where the keys are the metric names and the values are the metric scores;- a dictionary with metric names as keys and callables as values.See :ref:`multimetric_grid_search` for an example.If None, the estimator's score method is used.",'average_precision'
,"n_jobs n_jobs: int, default=NoneNumber of jobs to run in parallel.``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.``-1`` means using all processors. See :term:`Glossary `for more details... versionchanged:: v0.20 `n_jobs` default changed from 1 to None",-1
,"refit refit: bool, str, or callable, default=TrueRefit an estimator using the best found parameters on the wholedataset.For multiple metric evaluation, this needs to be a `str` denoting thescorer that would be used to find the best parameters for refittingthe estimator at the end.Where there are considerations other than maximum score inchoosing a best estimator, ``refit`` can be set to a function whichreturns the selected ``best_index_`` given the ``cv_results_``. In thatcase, the ``best_estimator_`` and ``best_params_`` will be setaccording to the returned ``best_index_`` while the ``best_score_``attribute will not be available.The refitted estimator is made available at the ``best_estimator_``attribute and permits using ``predict`` directly on this``RandomizedSearchCV`` instance.Also for multiple metric evaluation, the attributes ``best_index_``,``best_score_`` and ``best_params_`` will only be available if``refit`` is set and all of them will be determined w.r.t this specificscorer.See ``scoring`` parameter to know more about multiple metricevaluation.See :ref:`this example`for an example of how to use ``refit=callable`` to balance modelcomplexity and cross-validated score... versionchanged:: 0.20 Support for callable added.",True
,"cv cv: int, cross-validation generator or an iterable, default=NoneDetermines the cross-validation splitting strategy.Possible inputs for cv are:- None, to use the default 5-fold cross validation,- integer, to specify the number of folds in a `(Stratified)KFold`,- :term:`CV splitter`,- An iterable yielding (train, test) splits as arrays of indices.For integer/None inputs, if the estimator is a classifier and ``y`` iseither binary or multiclass, :class:`StratifiedKFold` is used. In allother cases, :class:`KFold` is used. These splitters are instantiatedwith `shuffle=False` so the splits will be the same across calls.Refer :ref:`User Guide ` for the variouscross-validation strategies that can be used he

In [30]:
best_params=search.best_params_
print(best_params)

{'subsample': 0.8, 'n_estimators': 300, 'max_depth': 7, 'learning_rate': 0.05, 'colsample_bytree': 0.8}


In [31]:
xgb_tuned=XGBClassifier(**best_params,eval_metric="logloss",random_state=42)
xgb_tuned.fit(X_train_res,y_train_res)
y_prob_xgb_tuned=xgb_tuned.predict_proba(X_test)[:,1]
pc_xgb_tuned=average_precision_score(y_test,y_prob_xgb_tuned)
print(pc_xgb_tuned)

0.879242129568531


In [32]:
with mlflow.start_run(run_name="XGBoost_tuned"):
    mlflow.log_params(best_params)
    mlflow.log_metric("pc_auc",pc_xgb_tuned)
    mlflow.xgboost.log_model(xgb_tuned,artifact_path="model")

2026/01/20 12:02:22 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🏃 View run XGBoost_tuned at: http://127.0.0.1:5000/#/experiments/1/runs/2dc0464cc5a046259be471efa8990d97
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1


In [33]:
precision,recall,thresholds=precision_recall_curve(y_test,y_prob_xgb_tuned)


In [34]:
alert_rate=0.01
business_threshold=np.percentile(y_prob_xgb_tuned,100*(1-alert_rate))
print("business_threshold:",business_threshold)

business_threshold: 0.014117929


In [35]:
y_final_pred=(y_prob_xgb_tuned>=business_threshold).astype(int)
print(classification_report(y_test,y_final_pred))

              precision    recall  f1-score   support

           0       1.00      0.99      1.00     56864
           1       0.16      0.91      0.27        98

    accuracy                           0.99     56962
   macro avg       0.58      0.95      0.63     56962
weighted avg       1.00      0.99      0.99     56962



In [36]:
with mlflow.start_run(run_name="Final_XGBoost"):
    mlflow.log_params(best_params)
    mlflow.log_param("threshold",business_threshold)
    mlflow.log_metric("pc_auc",pc_xgb_tuned)
    mlflow.xgboost.log_model(xgb_tuned,artifact_path="model")

2026/01/20 12:03:21 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🏃 View run Final_XGBoost at: http://127.0.0.1:5000/#/experiments/1/runs/bc78354529314d76a1395594cb28237d
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1


In [29]:
print("final model:XGBoost")
print("alertrate:",alert_rate)
print("threshold",business_threshold)

final model:XGBoost
alertrate: 0.01
threshold 0.0056028846


In [37]:
mlflow.set_experiment("Fraud_Detection")
mlflow.set_tracking_uri("http://127.0.0.1:5000")
model_name="Fixed_Model"
run_id=input("Enter run id:")
artifact_path="model"
model_uri=f"runs:/{run_id}/{artifact_path}"
result=mlflow.register_model(model_uri=model_uri,name=model_name)

Successfully registered model 'Fixed_Model'.
2026/01/20 12:06:22 WARNING mlflow.tracking._model_registry.fluent: Run with id bc78354529314d76a1395594cb28237d has no artifacts at artifact path 'model', registering model based on models:/m-d852f04eb357470e84fee6434848268c instead
2026/01/20 12:06:23 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: Fixed_Model, version 1
Created version '1' of model 'Fixed_Model'.
